In [1]:
import RNA

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import SVG, display

from collections import Counter
from collections import defaultdict
import subprocess
import difflib
import sys
import os
import random
import string
import time

sys.path.append('../')
from pretty_print_path import print_moves
import findpath_librna
import findpath


In [2]:
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True) # no scientific notation

# import feature_generation
from features import ij_distance, new_move_dist, plt_moves, config_distance, balance_in_all_things, return_shift_moves
from process_features import fp_call, find_moves, process

In [3]:
sequence = 'AGGUGUUAUAAAAGUGCAGAGCCAAGUCACCGCCUUGCCGUCGCUUCCGCUAUGCCUCAAAAGCGGAAUAUCGAGCGAGCGAUUGUGAUUCGAAUGGUCG'
s1       = '.(((................))).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........'
s2       = '...(((.........)))..((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..'

search_width = 60
# search_width = 500

s = s1

sw = 2
fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=sw, mp=True)
result = fp.get_en()/100.0
path = fp.get_path()


add_moves        = set()
delete_moves     = set()
available_add    = set()
available_delete = set()

en_contrib = []

for e, tup in enumerate(path):
    i = tup[0]
    j = tup[1]
    if i<0:
        delete_moves.add((i,j))
    if i>0:
        add_moves.add((i,j))

## we are evaluating 1 node = 1 structure
## means we are evaluating ALL moves (i, j) and see if we can find any patterns

##np.mean(en_contrib) ## mean of energies
##np.std(en_contrib) ## diversity of energies


## verhaeltnis delete add moves bezogen auf accessible moves
## mean of energy
## how many add / delete moves are currently available?
## if len(available_delete)/len(delete_moves) is high, the probability
## of making a wrong decision is probably quite high. 

## -------
## true values for classifier:
## - percentage of moves which lead to the optimal result?
## - sort by energy, where is the first contribution which leads to the optimal result? 
## - look at top energy contributions: how long do we have to postpone them to get the optimal result? 


fc = RNA.fold_compound(sequence)
pt = list(RNA.ptable(s))
pt1 = list(RNA.ptable(s1))
pt2 = list(RNA.ptable(s2))

for pos, (i,j) in enumerate(find_moves(pt, pt2)):  
    if (i,j) in add_moves:
        available_add.add((i,j))
    if (i,j) in delete_moves:
        available_delete.add((i,j))
    
    # map energies somehow to [0,1] such that the network will understand.  
    en = np.interp(fc.eval_move_pt(pt, i, j)/100, [-10,10], [0,1]) 
    en_contrib.append(en)

available_add, available_delete, len(available_add)/len(add_moves), len(available_delete)/len(delete_moves)

# np.std(en_contrib)

({(5, 17), (6, 16), (24, 95), (32, 83)},
 {(-43, -72),
  (-42, -73),
  (-41, -74),
  (-40, -76),
  (-39, -77),
  (-38, -79),
  (-37, -80),
  (-36, -81),
  (-35, -82),
  (-4, -21),
  (-3, -22),
  (-2, -23)},
 0.25,
 1.0)

In [4]:
# [   0,    0 ] .(((................))).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -23.10 0.125
# [  -4,  -21 ] .((..................)).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -20.70 0.1875
# [  -3,  -22 ] .(....................).((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -17.70 0.0
# [  -2,  -23 ] ........................((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))..........  -21.90 0.05882
# [  23,   96 ] ......................(.((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..)))))).....)....  -18.10 0.0
# [  22,   97 ] .....................((.((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..)))))).....))...  -21.00 0.0
# [  24,   95 ] .....................(((((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))....)))...  -23.60 0.0
# [  21,   98 ] ....................((((((((((....(((((((((.(((((((.(......).)))))))...))).)).))))..))))))....))))..  -25.20 0.07692
# [ -35,  -82 ] ....................((((((((((.....((((((((.(((((((.(......).)))))))...))).)).)))...))))))....))))..  -23.60 0.08333
# [ -36,  -81 ] ....................((((((((((......(((((((.(((((((.(......).)))))))...))).)).))....))))))....))))..  -22.70 0.1667
# [ -37,  -80 ] ....................((((((((((.......((((((.(((((((.(......).)))))))...))).)).).....))))))....))))..  -19.10 0.0
# [ -38,  -79 ] ....................((((((((((........(((((.(((((((.(......).)))))))...))).)).......))))))....))))..  -20.00 0.1667
# [ -39,  -77 ] ....................((((((((((.........((((.(((((((.(......).)))))))...))).)........))))))....))))..  -17.40 0.0
# [ -40,  -76 ] ....................((((((((((..........(((.(((((((.(......).)))))))...)))..........))))))....))))..  -19.70 0.0
# [  32,   83 ] ....................((((((((((.(........(((.(((((((.(......).)))))))...)))........).))))))....))))..  -18.10 0.0
# [  33,   81 ] ....................((((((((((.((.......(((.(((((((.(......).)))))))...)))......).).))))))....))))..  -17.10 0.0
# [  34,   79 ] ....................((((((((((.(((......(((.(((((((.(......).)))))))...)))....).).).))))))....))))..  -16.30 0.0
# [  35,   78 ] ....................((((((((((.((((.....(((.(((((((.(......).)))))))...)))...)).).).))))))....))))..  -17.90 0.0
# [  36,   77 ] ....................((((((((((.(((((....(((.(((((((.(......).)))))))...)))..))).).).))))))....))))..  -19.50 0.0
# [  37,   76 ] ....................((((((((((.((((((...(((.(((((((.(......).)))))))...))).)))).).).))))))....))))..  -21.50 0.0
# [  38,   75 ] ....................((((((((((.(((((((..(((.(((((((.(......).)))))))...)))))))).).).))))))....))))..  -24.60 0.0
# [ -41,  -74 ] ....................((((((((((.(((((((...((.(((((((.(......).)))))))...)).))))).).).))))))....))))..  -23.20 0.0
# [ -42,  -73 ] ....................((((((((((.(((((((....(.(((((((.(......).)))))))...)..))))).).).))))))....))))..  -19.90 0.0
# [ -43,  -72 ] ....................((((((((((.(((((((......(((((((.(......).)))))))......))))).).).))))))....))))..  -22.70 0.0
# [  39,   73 ] ....................((((((((((.((((((((.....(((((((.(......).)))))))....).))))).).).))))))....))))..  -21.90 0.0
# [  40,   72 ] ....................((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..  -25.20 0.0
# [   5,   17 ] ....(...........)...((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..  -21.20 0.0
# [   6,   16 ] ....((.........))...((((((((((.(((((((((....(((((((.(......).)))))))...)).))))).).).))))))....))))..  -23.70 0.0

In [7]:
def adjust_pt(pt, i, j):
    pt_adj = pt.copy()
    if i<0:
        pt_adj[-i] = 0
        pt_adj[-j] = 0
    else:
        pt_adj[i] = j
        pt_adj[j] = i
    return pt_adj

In [13]:
# which nodes do we permutate first? in principle, this is a simlar concept as the findpath search width, where we just
# use the sw. to perform a look-ahead

## IDEA
# check how many moves out of all choices are net-negative. 

search_width_multiplier = 2

sequence = 'CCUCUUGGCAAGGAACUGUUGUAUGACGGUACGUAACGCGCGCCAGAGCUCAAGAUUCAAACGGCGGCUUCAGAACUCAGUCAGGCGAAGAAGCCAGCAC'
s1       = '.......((.(.(.((((((....)))))).).)...))........................((((((((....((......))....)))))).))..'
s2       = '.(((.((((..(..((((((....)))))).).........)))))))...............((((((((....((......))....)))))).))..'

# sequence = 'GAGACCGAUACGAUUAAGCCAGAAUUCCAAUGAUUUGGGGAUCGGGGCUGUUUAGUCUCGUUUAAUCCCGGCGAUGUCUGGGUCAUAUUGUCCGUUAACC' 
# s1       = '(.((.(((((.(((....(((((..(((((....)))(((..(((((((....)))))))......)))...))..)))))))).)))))))).......'
# s2       = '.((((.((.((.....((((.((.((((((....)))))).))..)))).....)).))))))...(((((......)))))..................'

sequence = 'GACUGCCUCAAUUUCCACUUCGCCACAAUGUGAGGUCUCAUUCAACUGCCGCUCAUAAUGGUCGGUCAAUGAGGUCCAUCCGGUCCCUCCAGCCUGGCUA'
s1       = '.....................((((...((..((..((((((..((((((.........)).)))).))))))............).).))...))))..'
s2       = '.....................((((...((.((((.((...(((...((((((......)).))))...)))((.....)))).)).))))...))))..'



fc = RNA.fold_compound(sequence)
pt1 = list(RNA.ptable(s1))
pt2 = list(RNA.ptable(s2))

# for i,j in find_moves(pt1, pt2):    
#     pt = adjust_pt(pt1, i, j)
#     s = RNA.db_from_ptable(pt)
#     search_width = 1
#     sE, path = findpath_librna.pathfinder(sequence, s, s2, search_width=search_width, return_paths=True)
#     en = fc.eval_move_pt(pt1, i, j)
#     # print_moves(sequence, s, s2, path)
#     print (s, i, j, en, sE)

# def structure_evaluation(sequence, s, s2):
def structure_evaluation(sequence, pt1, pt2):
    avail_moves = []
    fp_results = []
    en_contrib = []

    available_add = set()
    available_delete = set()

    # pt1 = list(RNA.ptable(s))
    # pt2 = list(RNA.ptable(s2))
    for pos, (i,j) in enumerate(find_moves(pt1, pt2)):    
        next_en = fc.eval_move_pt(pt1, i, j)
        # mark where we found our move    
        # map energies somehow to [0,1] such that the network will understand.  
        en = np.interp(next_en/100, [-10,10], [0,1]) 
        en_contrib.append(en)
        avail_moves.append((i, j, next_en))

        if i>0:
            available_add.add((i,j))
        elif i<0:
            available_delete.add((i,j))

    if len(avail_moves)==0:
        add_delete = 0
    else:
        add_delete = len(available_delete)/len(avail_moves)

    en_mean = round(np.mean(en_contrib),2)
    en_std = round(np.std(en_contrib),2)

    en_contrib.sort(key=lambda x: x)
    en_diff = round(en_contrib[0] / en_mean, 3) # best en, relative

    # print ('best', en_contrib[0], 'avg', en_mean)

    avail_moves.sort(key=lambda x: x[2])

    # print ('avail add', available_add)
    # print ('avail del', available_delete)

    for pos, (i,j, en) in enumerate(avail_moves):  
        pt = adjust_pt(pt1, i, j)
        snew = RNA.db_from_ptable(pt)
        # if i > 0:
        #     snew = s[:i-1] + "(" + s[i:j-1] + ")" + s[j:]
        # if i < 0:
        #     snew = s[:-i-1] + "." + s[-i:-j-1] + "." + s[-j:]

        ptnew = list(RNA.ptable(snew))
        result_new, path = fp_call(sequence, snew, s2, search_width_multiplier)
        fp_results.append(result_new)

    if len(avail_moves) == 0:
        best_result = 0
    else:
        best_result = round(np.argmin(fp_results)/len(fp_results),2)
    
    # return best_result, en_mean, en_std, add_delete
    return best_result, en_mean, en_diff, avail_moves[0:3]

    print (avail_moves)
    print (fp_results)
    print (best_result)

# structure_evaluation(sequence, s, s2)
# structure_evaluation(sequence, pt1, pt2)

search_width = 500
# sE, path = findpath_librna.pathfinder(sequence, s1, s2, search_width=search_width, return_paths=True)
sE, path = fp_call(sequence, s1, s2, search_width_multiplier)
# en = fc.eval_move_pt(pt1, i, j)

ptlast = pt1.copy()
for i, j, en in path:
    if i==0: 
        continue
    # if i not in random_moves:
    #     continue

    pt = adjust_pt(ptlast, i, j)


    best_result, en_mean, en_std, add_delete = structure_evaluation(sequence, pt, pt2)

    # print (RNA.db_from_ptable(pt)[0:80], i, j, best_result, en_mean, en_std, add_delete)
    print (RNA.db_from_ptable(pt)[0:100], i, j, best_result, en_std, add_delete)
    
    ptlast = pt




# print_moves(sequence, s1, s2, path)


# while loop - increase sw from 1 onwards until we find the optimal path. how easy is the next decision?


.....................((((...((.(((..((((((..((((((.........)).)))).))))))............).))))...)))).. 32 89 0.0 0.485 [(35, 85, -350), (74, 80, 80), (-42, -68, 120)]
.....................((((...((.((((.((((((..((((((.........)).)))).))))))...........)).))))...)))).. 35 85 0.38 0.812 [(-42, -68, 120), (-45, -66, 180), (74, 80, 180)]
.....................((((...((.((((.((((((..(((((...........).)))).))))))...........)).))))...)))).. -50 -60 0.0 0.776 [(-49, -61, 40), (-42, -68, 120), (-45, -66, 180)]
.....................((((...((.((((.((((((..((((..............)))).))))))...........)).))))...)))).. -49 -61 0.0 0.788 [(52, 61, 40), (-42, -68, 120), (-48, -63, 170)]
.....................((((...((.((((.((((((..((((...(........).)))).))))))...........)).))))...)))).. 52 61 0.0 0.667 [(53, 60, -120), (-42, -68, 120), (-45, -66, 180)]
.....................((((...((.((((.((((((..((((...((......)).)))).))))))...........)).))))...)))).. 53 60 0.36 0.824 [(-42, -68, 120), (-45, -66, 180), (74, 80,

IndexError: list index out of range

In [65]:
sE, path = fp_call(sequence, s1, s2, search_width_multiplier)


In [6]:
filename_samples = f'./dataset_100_large.csv'
samples_df = pd.read_csv(filename_samples)
sequence = ''
s1 = ''
s2 = ''
# good example? 22 37

all_results = []

for index, row in samples_df.iterrows():
    # if index != 90:
    #     continue


    sequence = row.sequence
    s1 = row.s1
    s2 = row.s2


    bp_dist = RNA.bp_distance(s1, s2)
    # print (sequence)
    # print (s1)
    # print (s2)
    # print (bp_dist)

    # random_move = random.choice(range(bp_dist))

    fc = RNA.fold_compound(sequence)
    pt1 = list(RNA.ptable(s1))
    pt2 = list(RNA.ptable(s2))
    # structure_evaluation(sequence, pt1, pt2)

    sE, path = fp_call(sequence, s1, s2, search_width_multiplier)

    # print (path)
    path = path[0:-10] # ignore the last moves, we're not interested in meaningless decisions
    # print (path)

    random_moves = random.choices([i[0] for i in path], k=3)
    # print (random_moves)
    # print (len(path))

    ptlast = list(RNA.ptable(s1))

    all_evals = 0
    results = []

    # find a random structure in the path
    for i, j, en in path:
        if i==0: 
            continue
        # if i not in random_moves:
        #     continue

        pt = adjust_pt(ptlast, i, j)
        # print (i, RNA.db_from_ptable(pt))

        if i in random_moves:
            best_result, en_mean, en_std, add_delete = structure_evaluation(sequence, pt, pt2)
            results.append((sequence, RNA.db_from_ptable(pt), s2, best_result, en_mean, en_std, add_delete))
            all_evals += best_result       
            # print (i, eval)  

        ptlast = pt

    if all_evals != 0:
        all_results += results


    # break

    if index%10 == 0:        
        print (all_results)
        df = pd.DataFrame(all_results, columns=['sequence', 's', 's2', 'eval', 'en_mean', 'en_std', 'add_delete']).set_index('sequence')
        filename = f'./move_diversity.csv'
        df.to_csv(filename)

    if index == 50:
        break



best 0.38 avg 0.7 [0.38, 0.555, 0.6200000000000001, 0.655, 0.675, 0.6915, 0.6950000000000001, 0.6950000000000001, 0.7200000000000001, 0.7400000000000001, 0.75, 0.755, 0.79, 0.795, 0.81, 0.835]
best 0.4395 avg 0.58 [0.4395, 0.61, 0.6950000000000001]
best 0.5750000000000001 avg 0.72 [0.5750000000000001, 0.67, 0.671, 0.6900000000000001, 0.7000000000000001, 0.7050000000000001, 0.7290000000000001, 0.7515000000000001, 0.785, 0.8150000000000001, 0.8150000000000001]
[('CUGACUCCGGGUGCCCCGUACACCGAGACCAAGCUCACCCUCCGGAUCCAAGCAACUGUUCAGUCGAGAUGGGUCAUGGGACGGAGUGCACAGGCCAAUG', '(((((..........((((.(.(((.((((...(((.............................)))...)))).)))))))).))...))).......', '(((((((((((((.......))))(((......))).(((....((((((.........((......))))))))..))).))))))...))).......', 0.0, 0.7, 0.543, [(6, 85, -240), (-21, -80, 110), (-25, -77, 240)]), ('CUGACUCCGGGUGCCCCGUACACCGAGACCAAGCUCACCCUCCGGAUCCAAGCAACUGUUCAGUCGAGAUGGGUCAUGGGACGGAGUGCACAGGCCAAUG', '(((((((((((((.......))))...........(............